In [1]:
using Plots
gr(size=(600,400))
plotly()

Plots.PlotlyBackend()

In [2]:
include("../src/geompack.jl")

ProjOverHyperPlaneND (generic function with 1 method)

In [3]:
include("../src/MB.jl")
using MB

ArgumentError: ArgumentError: Package MB not found in current path:
- Run `import Pkg; Pkg.add("MB")` to install the MB package.


In [4]:
# this is the origin of the referencial
origin = VecND(3,[1,2,3]);
# this is the direction of the movement of the pendulum
direction = VecND(3,[1,1,2]);

npts = 2000
lower = 0 
upper = 10

println(origin)
println(direction)

VecND(3, [1.0, 2.0, 3.0])
VecND(3, [1.0, 1.0, 2.0])


In [5]:
# these are the scalars that together give the magnitude of the signal
multp = [cos(x) for x in range(lower,stop=upper,length=npts)];

In [6]:
x = range(lower,stop=upper,length=npts)
plot(x,multp)

In [7]:
# we add some noise to it
stdv = 0.1

bnoise = stdv * randn(npts);
observed = multp + bnoise;
plot(x,observed,label = "Observed Signal")
plot!(x,multp,w=3,label="Original Signal")

In [8]:
# the observations are made in a 3D space
observation_pts = map(x -> SomaND(origin,ProdByScalarND(x,direction)),observed);

ox = [observation_pts[i].values[1] for i in 1:npts];
oy = [observation_pts[i].values[2] for i in 1:npts];
oz = [observation_pts[i].values[3] for i in 1:npts];

In [9]:
# we add the random noise in each component
xnoise = 0.3 * randn(npts);
ynoise = 0.5 * randn(npts);
znoise = 0.2 * randn(npts);

noise_vectors = [SomaND(observation_pts[i],VecND(3,[xnoise[i],ynoise[i],znoise[i]])) for i in 1:npts];

In [10]:
ax = [noise_vectors[i].values[1] for i in 1:npts];
ay = [noise_vectors[i].values[2] for i in 1:npts];
az = [noise_vectors[i].values[3] for i in 1:npts];

In [11]:
gr(size=(900,600))
plotly()

Plots.PlotlyBackend()

In [12]:
scatter3d(ax,ay,az,markersize=.6,label = "Observations")
scatter3d!(ox,oy,oz,markersize = .6 , label = "Original Data")

In [13]:
pt1 = VecND(3,[2,-10,5])
vec11 = VecND(3,[-1,2,1])
vec12 = VecND(3,[1,-10,-3])
pl1varray = [vec11,vec12]
plane1 = HyperPlaneNDInit(pt1,pl1varray)

pt2 = VecND(3,[-5.,10,-3])
vec21 = VecND(3,[-0.042486352066674525, -0.7748369397560202, 0.32968505000961074])
vec22 = VecND(3,[1,-10,-3])
pl2varray = [vec21,vec22]
plane2 = HyperPlaneNDInit(pt2,pl2varray)


pt3 = VecND(3,[-2.403287986454733, -4.1295977249862, 0.68818196014408])
vec31 = VecND(3,[-10,2,-2])
vec32 = VecND(3,[0.4750280825480982, -0.0368312710814023, -0.592734463002089])
pl3varray = [vec31,vec32]
plane3 = HyperPlaneNDInit(pt3,pl3varray);

In [14]:
dataview1 = map(x-> ProjOverHyperPlaneND(x,plane1), noise_vectors);
dataview2 = map(x-> ProjOverHyperPlaneND(x,plane2), noise_vectors);
dataview3 = map(x-> ProjOverHyperPlaneND(x,plane3), noise_vectors);

In [15]:
ax1 = [dataview1[i].values[1] for i in 1:npts];
ay1 = [dataview1[i].values[2] for i in 1:npts];
az1 = [dataview1[i].values[3] for i in 1:npts];

ax2 = [dataview2[i].values[1] for i in 1:npts];
ay2 = [dataview2[i].values[2] for i in 1:npts];
az2 = [dataview2[i].values[3] for i in 1:npts];

ax3 = [dataview3[i].values[1] for i in 1:npts];
ay3 = [dataview3[i].values[2] for i in 1:npts];
az3 = [dataview3[i].values[3] for i in 1:npts];

In [16]:
scatter3d(ax,ay,az,markersize=.6,label = "Observations")
scatter3d!(ax1 , ay1 , az1 , markersize = .6 , label = "Data View 1")
scatter3d!(ax2 , ay2 , az2 , markersize = .6 , label = "Data View 2")
scatter3d!(ax3 , ay3 , az3 , markersize = .6 , label = "Data View 3")

In [17]:
# plane1
# we create the vectors in the orthogonal base 
versor11 = VersorND(vec11)
versor12 = VectorProdND([versor11,plane1.normal])

proj2Dplane1 = map(x-> VecND(2,
        [
        DotND(SomaND(x,ProdByScalarND(-1.0,plane1.point)),versor11),
        DotND(SomaND(x,ProdByScalarND(-1.0,plane1.point)),versor12)
            ]
    ),dataview1
);

pl1x = [proj2Dplane1[i].values[1] for i in 1:npts];
pl1y = [proj2Dplane1[i].values[2] for i in 1:npts];

plot1 = scatter(pl1x,pl1y,markersize = .8,label="Data view in plane 1");

In [18]:
# plane2
# we create the vectors in the orthogonal base 
versor21 = VersorND(vec21)
versor22 = VectorProdND([versor21,plane2.normal])

proj2Dplane2 = map(x-> VecND(2,
        [
        DotND(SomaND(x,ProdByScalarND(-1.0,plane2.point)),versor21),
        DotND(SomaND(x,ProdByScalarND(-1.0,plane2.point)),versor22)
            ]
    ),dataview2
);

pl2x = [proj2Dplane2[i].values[1] for i in 1:npts];
pl2y = [proj2Dplane2[i].values[2] for i in 1:npts];

plot2 = scatter(pl2x,pl2y,markersize = .8,label="Data view in plane 1");

In [19]:
# plane3
# we create the vectors in the orthogonal base 
versor31 = VersorND(vec31)
versor32 = VectorProdND([versor31,plane3.normal])

proj2Dplane3 = map(x-> VecND(2,
        [
        DotND(SomaND(x,ProdByScalarND(-1.0,plane3.point)),versor31),
        DotND(SomaND(x,ProdByScalarND(-1.0,plane3.point)),versor32)
            ]
    ),dataview3
);

pl3x = [proj2Dplane3[i].values[1] for i in 1:npts];
pl3y = [proj2Dplane3[i].values[2] for i in 1:npts];

plot3 = scatter(pl3x,pl3y,markersize = .8,label="Data view in plane 1");

In [20]:
plot(plot1,plot2,plot3,layout=(3,1),legend = false,aspect_ratio = 1 ,size = (400,1200))

In [21]:
matrix1 = VecNDToMatrix(proj2Dplane1);
matrix2 = VecNDToMatrix(proj2Dplane2);
matrix3 = VecNDToMatrix(proj2Dplane3);


matrix = cat(matrix1 , matrix2 , matrix3, dims= 2);

In [22]:
svd = MB.pca(matrix);
pca = MB.principal_components(svd,1)
reduxpca = MB.reduce_dimensionality(svd,1);

MethodError: MethodError: no method matching reduce_dimensionality(::SVD{Float64,Float64,Array{Float64,2}}, ::Int64)
Closest candidates are:
  reduce_dimensionality(::SVD; k) at /mnt/juliabox/MB/src/MB.jl:21